<a href="https://colab.research.google.com/github/sibot89/PyTorch/blob/main/Binary_Prediction_of_Poisnous_Mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim import SGD

In [2]:
torch.__version__

'2.6.0+cu124'

In [28]:
!mkdir /content/data

mkdir: cannot create directory ‘/content/data’: File exists


In [29]:
!cp /content/drive/MyDrive/data/playground-series-s4e8.zip /content/data/

In [31]:
!unzip /content/data/playground-series-s4e8.zip -d /content/data

Archive:  /content/data/playground-series-s4e8.zip
replace /content/data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!rm /content/data/playground-series-s4e8.zip

In [12]:
df_train = pd.read_csv('/content/data/train.csv')
df_train

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [13]:
df_train = df_train.drop(['id'], axis=1)
df_train.shape

(3116945, 21)

In [17]:
num_samples, num_features = df_train.shape
num_features -= 1
num_classes = len(np.unique(df_train['class']))
num_samples, num_features, num_classes

(3116945, 20, 2)

In [ ]:
df_train, df_valid = train_test_split(df_train, train_size=0.8,
                                                    random_state=42, shuffle=True, stratify=y)


Dealing with missin values

In [21]:
df_train.isna().mean() * 100

,0
class,0.000000
cap-diameter,0.000128
cap-shape,0.001283
cap-surface,21.528227
cap-color,0.000385
does-bruise-or-bleed,0.000257
gill-attachment,16.809280
gill-spacing,40.373988
gill-color,0.001829
stem-height,0.000000


In [26]:
missing_threshold = 0.95

high_missing_threshold = df_train.columns[df_train.isna().mean() >
                                          missing_threshold]

df_train = df_train.drop(columns=high_missing_threshold)
df_valid = df_valid.drop(columns=high_missing_threshold)

NameError: name 'df_valid' is not defined

In [ ]:
mode_values = df_train.mode()
mode_values = mode_values.loc[0].to_dict()

median_values = df_train.median(numeric_only=True).to_dict()

In [ ]:
for